In [1]:
import pandas as pd
import numpy as np
import regex as re
import pandas as pd
import emoji
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from bltk.langtools import Tokenizer
from bltk.langtools import UgraStemmer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from bangla_stemmer.stemmer import stemmer
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [12]:
pip install regexp

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for regexp: filename=regexp-0.1-py3-none-any.whl size=1432 sha256=76a2f7eaac76232f51a3ce294989707ffe4e23dbd6214af6d40178cb5ecf2a59
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\d8\37\7d\55cbeb7fcea2a9e1e1a48de7ba53b35f5b68994d7b67db4c55
Successfully built regexp
Note: you may need to restart the kernel to use updated packages.


In [3]:
dataset = pd.read_csv('younus.csv')
#dataset = pd.read_excel('data.xlsx')
dataset = dataset.dropna()
#dataset = dataset[0:5000]
#dataset = dataset[['comment','label']]
print(len(dataset))

2011


In [4]:
dataset

,Unnamed: 0,Review,Sentiment
0,0,বই টি আমার সংগ্রহে আছে এবং তা আমি পড়েছি। বই টি...,0
1,1,ফালতু বই। শুধু শুধু টাকা নষ্ট।,0
2,2,"একটু পড়ে দেখতে বলসিলো, একটু পড়লাম। তারপর হড়হড় ...",0
3,3,এটা কোন বই? ছিঃ ধিকার জানায়। প্রশাসনের নজর দেয়...,0
4,4,"প্রিয় রকমারি.কম, দয়া করে বইটি আপনাদের ওয়েবসাইট...",0
...,...,...,...
2006,2006,1 স্টার দেওয়াও বৃথা। মানে ভিডিও বানাইয়া আজকে ...,0
2007,2007,তাদের কথা শুনলে মনে হয় তারা এই মনে হয় আশে পাশে...,0
2008,2008,এইরকম বই পড়ার চাইতে মুড়ি কিন্না খাওয়াও ভালো। ...,0
2009,2009,বোগাস বই। টাকা নষ্ট।\n ...,0


In [5]:
bangla_stop_words =["অতএব", "অথচ", "অথবা", "অনুযায়ী", "অনেক", "অনেকে", "অনেকেই", "অন্তত", "অবধি", "অবশ্য", 
"আগামী", "আগে", "আগেই", "আছে", "আজ", "আদ্যভাগে", "আপনার", "আপনি", "আবার", "আমরা", 
"আমাকে", "আমাদের", "আমার", "আমি", "আর", "আরও", "ইত্যাদি", "ইহা", "উচিত", "উনি", "উপর", 
"উপরে", "এ", "এঁদের", "এঁরা", "এই", "এক", "একই", "একজন", "একটা", "একটি", "একবার", "একে", 
"এখন", "এখনও", "এখানে", "এখানেই", "এটা", "এটাই", "এটি", "এত", "এতটাই", "এতে", "এদের", "এবং", 
"এবার", "এমন", "এমনকি", "এর", "এরা", "এলো", "এস", "এসে", "ঐ", "ও", "ওঁদের", "ওঁর", "ওঁরা", 
"ওই", "ওকে", "ওখানে", "ওদের", "ওর", "ওরা", "কখনও", "কত", "কথা", "কবে", "কয়েক", "কয়েকটি", 
"করছে", "করছেন", "করতে", "করবে", "করবেন", "করলে", "করলেন", "করা", "করাই", "করায়", "করার", 
"করি", "করিতে", "করিয়া", "করিয়ে", "করে", "করেই", "করেছিলেন", "করেছে", "করেছেন", "করেন", "কাউকে", 
"কাছ", "কাছে", "কাজ", "কাজে", "কারও", "কারণ", "কি", "কিংবা", "কিছু", "কিছুই", "কিন্তু", "কী", "কে", 
"কেউ", "কেউই", "কেন", "কোন", "কোনও", "কোনো", "ক্ষেত্রে", "খুব", "গিয়ে", "গিয়েছে", "গুলি", "গেছে", 
"গেল", "গেলে", "গোটা", "চলে", "চান", "চায়", "চেয়ে", "ছাড়া", "ছাড়াও", "ছিল", "ছিলেন", "জন", "জনকে", 
"জনের", "জন্য", "জন্যে", "জানতে", "জানা", "জানানো", "জানায়", "জানিয়ে", "জানিয়েছে", "টি", "তখন", 
"তত", "তথা", "তবু", "তবে", "তা", "তাঁকে", "তাঁদের", "তাঁর", "তাঁরা", "তাঁহারা", "তাই", "তাও", "তাকে", "তাতে", 
"তাদের", "তার", "তারপর", "তারা", "তারই", "তাহলে", "তাহা", "তাহাতে", "তাহার", "তিনই", "তিনি", "তিনিও", 
"তুমি", "তুলে", "তেমন", "তো", "তোমার", "থাকবে", "থাকবেন", "থাকা", "থাকায়", "থাকে", "থাকেন", "থেকে", 
"থেকেই", "থেকেও", "দিকে", "দিতে", "দিয়ে", "দিয়েছে", "দিয়েছেন", "দিলেন", "দিয়ে", "দু", "দুটি", "দুটো", 
"দেওয়া", "দেওয়ার", "দেখতে", "দেখা", "দেখে", "দেন", "দেয়", "দেশের", "দ্বারা", "ধরা", "ধরে", 
"নাকি", "নাগাদ", "নানা", "নিজে", "নিজেই", "নিজেদের", "নিজের", "নিতে", "নিয়ে", "নিয়ে", "নেওয়া", 
"নেওয়ার", "পক্ষে", "পর", "পরে", "পরেই", "পরেও", "পর্যন্ত", "পাওয়া", "পারি", "পারে", "পারেন", "পেয়ে", 
"প্রতি", "প্রভৃতি", "প্রায়", "ফলে", "ফিরে", "ফের", "বছর", "বদলে", "বরং", "বলতে", "বলল", "বললেন", "বলা", "বলে", 
"বলেছেন", "বলেন", "বসে", "বহু", "বা", "বাদে", "বার", "বিনা", "বিভিন্ন", "বিশেষ", "বিষয়টি", "বেশ", "ব্যবহার", 
"ব্যাপারে", "ভাবে", "ভাবেই", "মত", "মতো", "মতোই", "মধ্যভাগে", "মধ্যে", "মধ্যেই", "মধ্যেও", "মনে", "মাত্র", "মাধ্যমে", 
"মানুষ", "মানুষের", "মোট", "মোটেই", "যখন", "যত", "যতটা", "যথেষ্ট", "যদি", "যদিও", "যা", "যাঁর", "যাঁরা", "যাওয়া", 
"যাওয়ার", "যাকে", "যাচ্ছে", "যাতে", "যাদের", "যান", "যাবে", "যায়", "যার", "যারা", "যায়", "যিনি", "যে", "যেখানে", 
"যেতে", "যেন", "যেমন", "রকম", "রয়েছে", "রাখা", "রেখে", "শুধু", "শুরু", "সঙ্গে", "সঙ্গেও", "সব", "সবার", "সমস্ত", 
"সম্প্রতি", "সময়", "সহ", "সহিত", "সাথে", "সুতরাং", "সে", "সেই", "সেখান", "সেখানে", "সেটা", "সেটাই", "সেটাও", "সেটি", 
"স্পষ্ট", "স্বয়ং", "হইতে", "হইবে", "হইয়া", "হওয়া", "হওয়ায়", "হওয়ার", "হচ্ছে", "হত", "হতে", "হতেই", "হন", "হবে", 
"হবেন", "হয়", "হয়তো", "হয়নি", "হয়ে", "হয়েই", "হয়েছিল", "হয়েছে", "হয়েছেন", "হল", "হলে", "হলেই", "হলেও", "হলো", 
"হিসাবে", "হিসেবে", "হৈলে", "হোক", "হয়", "হয়ে", "হয়েছে", "অর্থাৎ", "অন্য", "অনুযায়ী", "অর্ধভাগে", "এসো", "কয়েক", 
"কয়েকটি", "করিয়ে", "করিয়া", "করায়", "গিয়ে", "গিয়েছে", "ছাড়া", "ছাড়াও", "থাকায়", "দিয়েছে", "দিয়েছেন", "দেয়", "দেওয়া", 
"দেওয়ার", "নেওয়ার", "পাওয়া", "পেয়ে", "প্রায়", "বিষয়টি", "যাওয়া", "যাওয়ার", "রয়েছে", "স্বয়ং", "হৈতে", "হইয়া", "হয়েছিল", 
"হয়েছেন", "হয়নি", "হয়েই", "হয়তো", "হওয়া", "হওয়ার", "হওয়ায়", "জানায়", "জাানিয়ে", "জানিয়েছে", "চায়", "চেয়ে", "উত্তর", 
"এমনি", "কেমনে", "কোটি", "চার", "চালু", "চেষ্টা", "দিন", "দুই", "নতুন", "পাঁচ", "প্রথম", "প্রাথমিক", "বক্তব্য", "বন", "বেশি", 
"লক্ষ", "সাধারণ", "সামনে", "হাজার"]
names = ['বাংলাদেশের','ভারত','বাংলাদেশ','তাহসান','মিথিলা','সৃজিত','তাহসানের',
        'আয়রা','আব্দুল লতিফ','আরে','তাহসিন','শাকিব','জয়','রুবেল','দিঘীকে',
         'দিঘি','দীঘির','দিগি','জোলিল','জলিল','অনন্ত','শাকিব','খান','তাহসিরেশন',
         'অপুকে','অপুর','সাকিব','অপু','নাহিদ','আজহারি','জলিল','জিহাদি',
        'শ্রীজিৎ','অনন্ত ','','','','','','']
stop_words_for_own_dataset = ['আ‌মি','ভারতে','দে‌খছি','তোর','কারন',
                              'বুঝা','একদিন','মানে','গেলো','হুম','রাখে',
                              'কেমন','ই','টু','তো','কেমন','ওটা',
                              'মেয়েটাকে','মার','স্বামীর','নাম','তোমরা','দাদা',
                              'মহিলা','টা','চাই','টাও','র','টা','এদেরকে','র্থ',
                              'ভাই','ভাইয়া','হজুর','আপ্নে','এইটা','ু','তারে',
                              'তুই','তাসনিম','রে','ন','তুমার','তোমারে','কইরো',
                              'আপনাকে','নকে','ল','থা','র','লোক','ব','জয়ের',
                              'র','ভাইয়া','ভারত','ধর্ষকের','পোশাক','ধর্ষন',
                              'ধর্ষনের','ধর্ষন','ধর্ষক','','','']

In [14]:
corpus = []
#stemmer = UgraStemmer()
stmr = stemmer.BanglaStemmer()
for index,value in dataset.iterrows():
    text = str(value['Review'])
    text = re.sub(r'https?:\/\/\S+','',text) #removing hyperlinks
    text = re.sub("[\.\,\!\?\:\;\-\=।*/+-@#$%^&()]", " ", text)
    text = re.sub("[A-Za-z]"," ",text)
    text = re.sub("[0-9]"," ",text)
    text = re.sub("[০-৯]"," ",text)
    text = text.replace("'", "")
    text = text.replace('"', '')
    text = emoji.get_emoji_regexp().sub(r'', text)
      
    temp = []
    tokens = Tokenizer().word_tokenizer(text)
    for word in tokens:
        if word not in bangla_stop_words:
            if word not in stop_words_for_own_dataset:
                if word not in names:
                    temp.append(word)
    
    text = ' '.join(temp)
    
#     temp = []
#     tokens = Tokenizer().word_tokenizer(text)
#     for word in tokens:
#         stm = stmr.stem(word)
#         temp.append(stm)
#     text = ' '.join(temp)
    
#     tokens = Tokenizer().word_tokenizer(text)
#     stem = stemmer.stem(tokens)
#     text = ' '.join(stem)
    
    text = text.strip()
    corpus.append(text)

AttributeError: module 'emoji' has no attribute 'get_emoji_regexp'

In [ ]:
corpus

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(corpus).toarray()

In [ ]:
y = dataset['Sentiment'].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 1)

In [ ]:
x_train.shape

In [ ]:
o = 0
for i in y_test:
    if i == 1:
        o = o + 1
print(o)

In [ ]:
## multinomial naive bayes for tf-idf

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
ann = Sequential()

In [ ]:
#Add the input layer and the first hidden layer
ann.add(Dense(units = 64,activation='relu',input_dim = 1716))
#Add the second hidden layer
ann.add(Dense(units = 64,activation='relu'))
#Add the output layer
ann.add(Dense(units = 1,activation='sigmoid'))

In [ ]:
#Compile the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.summary()

In [ ]:
ann.fit(x_train,y_train,validation_split=.3,batch_size=100,epochs=100)

In [ ]:
yp = ann.predict(x_test)
yp[:5]

In [ ]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

In [ ]:
x_train.shape

In [ ]:
# For conv1D dimentionality should be 187X1 where 187 is number of features and 1 = 1D Dimentionality of data
x_train = x_train.reshape(len(x_train),x_train.shape[1],1)
x_test = x_test.reshape(len(x_test),x_test.shape[1],1)

In [ ]:
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense # for fully connected layers dense will be used
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam

# avoid overfitting by normalizing the samples
from tensorflow.keras.layers import BatchNormalization
#from keras.layers.normalization import BatchNormalization

In [ ]:
def build_model():
    model = Sequential()
    
    # Filters = Units in Dense Total number of Neurons
    # Padding = 'same' , zero-padding, Add zero pixels all around input data
    model.add(Conv1D(filters = 64, kernel_size = 6, activation='relu', padding = 'same', input_shape = (1716, 1))) #we pass individual values hence not 100000,187,1
    
    # Normalization to avoid overfitting
    model.add(BatchNormalization())
    
    # Pooling 
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    model.add(Conv1D(filters = 64, kernel_size = 6, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    model.add(Conv1D( filters = 64, kernel_size = 6, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=(3), strides = (2), padding = 'same'))

    # Flatten 
    model.add(Flatten())

    # Fully connected layer
    # input layer
    model.add(Dense(units = 20, activation='relu'))
    
    # Hidden Layer
    model.add(Dense(units = 20, activation='relu'))
    
    # Output Layer
    model.add(Dense(units = 1, activation='softmax'))

    # loss = 'categorical_crossentropy'
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
model = build_model()

In [ ]:
# This is for one sample, i.e. one row
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs = 30, batch_size = 20, validation_data=(x_test, y_test))

In [ ]:
# evaluate ECG Test Data
model.evaluate(x_test, y_test)

In [ ]:
#Importing the libraries  
import numpy as nm  
import matplotlib.pyplot as mtp  
import pandas as pd 

In [ ]:
# Splitting the dataset into the Training set and Test set  
from sklearn.model_selection import train_test_split  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 1)  


In [ ]:
 #Feature Scaling  
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)